In [1]:
import pickle
import torch
import sys, os
import cv2 as cv
import h5py
import copy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import transformations as tf
import numpy as np
from itertools import repeat

np.set_printoptions(suppress=True)

%load_ext autoreload
%autoreload 2

In [2]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from policy import EPACTPolicy, ACTPolicy, DINOPolicy
from brl_constants import TASK_CONFIGS
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats
from utils import load_data, load_data_fix_val # data functions

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/zfei/code/robomimic/robomimic/scripts/setup_macros.py
)


In [3]:
ckpt_path = "/mnt/data1/act/train_act_scara_3cam/3cam_epact_v4_1_lep10/"
config_path = ckpt_path + "config.pkl"

with open(config_path, "rb") as f:
    policy_config = pickle.load(f)['policy_config']
    policy_config["ep_weight"] = 1.0

act_policy_config = copy.copy(policy_config)
dino_policy_config = copy.copy(policy_config)

act_policy_config['hidden_dim'] = 384
policy_class = "DINOACT"

In [4]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

val_episode_ids = [0, 1]
val_episode_len = [all_episode_len[i] for i in val_episode_ids]

chunk_size = 64


Found 51 hdf5 files


In [13]:
dataset_dir = "/mnt/data1/act/train_act_scara_3cam"
DATA_DIR = '/mnt/data1/act'

TASK_CONFIGS = {
        'epact':{
        'dataset_dir': dataset_dir + '/train_act_scara_3cam',
        'num_episodes': 100,
        'episode_len': 300,
        'camera_names': ['wrist', 'wrist_down']
    }
}

In [14]:
task_config = TASK_CONFIGS['epact']
name_filter = task_config.get('name_filter', lambda n: True)
batch_size_train = 8
batch_size_val = 8


In [15]:
chunk_size = 100
skip_mirrored_data = False
load_pretrain = False
train_ratio = 0.99
resize = None
resize = (518, 518)
print("dataset_dir: ", dataset_dir)
train_dataloader,val_dataloader, stats, _ = load_data_fix_val(dataset_dir, 
                                                       name_filter, 
                                                       camera_names, 
                                                       batch_size_train, 
                                                       batch_size_val, 
                                                       chunk_size,
                                                       skip_mirrored_data,
                                                       load_pretrain,
                                                       policy_class,
                                                       stats_dir_l=None,
                                                       sample_weights=None,
                                                       train_ratio=train_ratio,
                                                       resize=resize)


dataset_dir:  /mnt/data1/act/train_act_scara_3cam
Found 51 hdf5 files


Data from: ['/mnt/data1/act/train_act_scara_3cam']
- Train on [50] episodes
- Test on [1] episodes


Found 51 hdf5 files
Norm stats from: ['/mnt/data1/act/train_act_scara_3cam']
train_episode_len: [114, 175, 120, 154, 135, 135, 124, 104, 168, 98, 118, 121, 121, 136, 128, 125, 165, 136, 117, 140, 117, 139, 151, 123, 153, 124, 107, 110, 135, 104, 103, 161, 168, 129, 154, 111, 137, 177, 128, 101, 127, 118, 106, 116, 144, 110, 118, 108, 90, 132], val_episode_len: [116], train_episode_ids: [20 25 36 40  8 31 28 47 34  7 33 46  2 17  4 18 49 14 48 50 35 38 32 43
 41 23  6 26 29  3 30 37 19  9 10 12  0 44 16 27  1 13 22 42 45 24 11  5
 21 15], val_episode_ids: [39]
val data:  /mnt/data1/act/train_act_scara_3cam/data1/episode_13.hdf5
augment_images:  False
Initializing transformations
augment_images:  False
Initializing transformations
Augment images: False, train_num_workers: 8, val_num_workers: 8


In [17]:
data = next(train_dataloader)
image_data, qpos_data, action_data, is_pad = data





[DEBUG]image torch.Size([8, 3, 3, 518, 518])
[DEBUG]image2 torch.Size([8, 3, 3, 518, 518])
image torch.Size([8, 3, 3, 518, 518])
qpos torch.Size([8, 4])
actions torch.Size([8, 100, 5])
is_pad torch.Size([8, 100])
[DETRVAE]: cam_id: 0, image: torch.Size([8, 3, 518, 518])
DinoBackbone x:  torch.Size([8, 384, 1369])
DinoBackbone x:  torch.Size([8, 384, 375])
DinoBackbone x:  torch.Size([8, 384, 15, 25])
DinoJoiner x:  torch.Size([8, 384, 15, 25])
[DETRVAE]: cam_id: 0, cam_features: torch.Size([8, 384, 15, 25]), pos: torch.Size([1, 384, 15, 25])
[DETRVAE]: cam_id: 1, image: torch.Size([8, 3, 518, 518])
DinoBackbone x:  torch.Size([8, 384, 1369])
DinoBackbone x:  torch.Size([8, 384, 375])
DinoBackbone x:  torch.Size([8, 384, 15, 25])
DinoJoiner x:  torch.Size([8, 384, 15, 25])
[DETRVAE]: cam_id: 1, cam_features: torch.Size([8, 384, 15, 25]), pos: torch.Size([1, 384, 15, 25])
[DETRVAE]: cam_id: 2, image: torch.Size([8, 3, 518, 518])
DinoBackbone x:  torch.Size([8, 384, 1369])
DinoBackbone x:

{'l1': tensor(0.4779, device='cuda:0', grad_fn=<MeanBackward0>),
 'kl': tensor(8.3777, device='cuda:0', grad_fn=<SelectBackward0>),
 'loss': tensor(84.2548, device='cuda:0', grad_fn=<AddBackward0>)}